In [1]:
from retriever import Retriever, Ranker
from encoder import BoWEncoder
from indexer import RoutingNetwork
from tokenizer import Tokenizer
from utils import get_dataset
import torch

In [2]:
queries_path = "./dev_queries/raw.tsv"
queries = get_dataset(queries_path)

In [3]:
tokenizer = Tokenizer.load_state("./data/tokenizer.pkl")

In [4]:
encoder = BoWEncoder(
        embed_dim=tokenizer.vocab_size, vocab_size=tokenizer.vocab_size
    )

In [5]:
indexer = RoutingNetwork.load_state("./data/index.pkl")

In [6]:
q_text = queries[0]["text"]

In [7]:
print(q_text)

who sang i feel wonderful tonight


In [8]:
q_tokens = torch.tensor(tokenizer.encode(q_text))
q_embeds = encoder(q_tokens)

In [45]:
network = indexer.network

In [46]:
terms_sum = network.sum(-1)

In [47]:
terms_sum.unsqueeze(1).size()

torch.Size([30933, 1])

In [48]:
network[:, q_embeds.bool()]

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])

In [49]:
tf = network[:, q_embeds.bool()] / terms_sum.unsqueeze(1)

In [50]:
mask = network.bool() & q_embeds.bool()

In [51]:
num_docs_with_term = mask[:, q_embeds.bool()].int().sum(0)

In [52]:
num_docs_with_term 

tensor([ 367, 1266,   50,  225, 1515,  298])

In [53]:
idf = indexer.d_size * (1 / num_docs_with_term)

In [54]:
idf

tensor([ 84.2861,  24.4336, 618.6600, 137.4800,  20.4178, 103.8020])

In [55]:
tf.size()

torch.Size([30933, 6])

In [56]:
tf_idf = tf * idf

In [57]:
tf_idf = tf_idf.sum(-1)

In [67]:
vals, idx = tf_idf.topk(1000)